In [1]:
# import transformers
# import torch

# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# pipeline = transformers.pipeline(
#   "text-generation",
#   model="meta-llama/Meta-Llama-3-8B-Instruct",
#   model_kwargs={"torch_dtype": torch.bfloat16},
#   device="cuda",
# )
# run above to download and load models
# models are saved in cd ~/.cache/huggingface/hub/a

/home/yuxiawang/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:01<00:00,  3.49it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# LLaMA3 input format
<|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>
    {{ system_prompt }}<|eot_id|>
    <|start_header_id|>user<|end_header_id|>
    {{ user_msg_1 }}<|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    {{ model_answer_1 }}<|eot_id|>

In [ ]:
<|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>
    You are a mental health supporter, you'd like to help others.<|eot_id|>
    <|start_header_id|>user<|end_header_id|>
        I feel so sad, I lost my dog.<|eot_id|>

### VLLM inference

In [1]:
import vllm
import torch
import transformers

model_name="meta-llama/Meta-Llama-3-8B-Instruct"
model = vllm.LLM(
    model=model_name,
    tokenizer=model_name,
    tensor_parallel_size=2,
    dtype="bfloat16" if torch.cuda.is_bf16_supported() else "float16"
)

/home/yuxiawang/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-01 12:44:39,169	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-01 12:44:41,447	INFO worker.py:1749 -- Started a local Ray instance.


INFO 05-01 12:44:41 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='meta-llama/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 05-01 12:44:45 utils.py:608] Found nccl from library /home/yuxiawang/.config/vllm/nccl/cu12/libnccl.so.2.18.1
(RayWorkerWrapper pid=10165) INFO 05-01 12:44:45 utils.py:608] Found nccl from library /home/yuxiawang/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 05-01 12:44:45 selector.py:28] Using FlashAttention backend.
(RayWorkerWrapper pid=10165) INFO 05-01 12:44:45 selector.py:28] Using FlashAttention backend.
INFO 05-01 12:44:45 pynccl_utils.py:43] vLLM is using nccl==2.18.1
(RayWorkerWrapper pid=10165) INFO 05-01 12:44:45 pynccl_utils.py:43] vLLM is using nccl==2.18.1
INFO 05-01 12:44:45 utils.py:129] reading GPU P2P access cache from /home/yuxiawang/.config/vllm/gpu_p2p_access_cache_for_0,1.json
WARNING 05-01 12:44:45 custom_all_reduce.py:74] Custom allreduce is disabled because your platform lacks GPU P2P capability or P2P test failed. To silence this warning, specify disable_custom_all_reduce=True explicitly.
(RayWorkerWrapper pid=10165) INFO 05-01 12:44:45 utils.py:129] rea

[rank0]:[W CUDAGraph.cpp:145] Warning: Waiting for pending NCCL work to finish before starting graph capture. (function operator())
(RayWorkerWrapper pid=10165) [rank1]:[W CUDAGraph.cpp:145] Warning: Waiting for pending NCCL work to finish before starting graph capture. (function operator())


INFO 05-01 12:45:03 model_runner.py:1057] Graph capturing finished in 3 secs.
(RayWorkerWrapper pid=10165) INFO 05-01 12:45:03 model_runner.py:1057] Graph capturing finished in 3 secs.


In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
# print(tokenizer.chat_template)
# set generation parameters
sampling_params = vllm.SamplingParams(max_tokens=256)
print(sampling_params)

system_prompt = """You are a mental health supporter, you'd like to help others."""
samples = ["I feel so sad, I lost my dog.", "I was promoted to manager recently, but my colleagues seem not happy."]
model_inputs = [
    tokenizer.apply_chat_template(
        [
            {"role":"system", "content": system_prompt.strip()}, 
            {"role":"user", "content": sample.strip()}
        ], 
        tokenize=False)
    for sample in samples
]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


SamplingParams(n=1, best_of=1, presence_penalty=0.0, frequency_penalty=0.0, repetition_penalty=1.0, temperature=1.0, top_p=1.0, top_k=-1, min_p=0.0, seed=None, use_beam_search=False, length_penalty=1.0, early_stopping=False, stop=[], stop_token_ids=[], include_stop_str_in_output=False, ignore_eos=False, max_tokens=256, min_tokens=0, logprobs=None, prompt_logprobs=None, skip_special_tokens=True, spaces_between_special_tokens=True, truncate_prompt_tokens=None)


In [3]:
print(model_inputs[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a mental health supporter, you'd like to help others.<|eot_id|><|start_header_id|>user<|end_header_id|>

I feel so sad, I lost my dog.<|eot_id|>


In [6]:
result = model.generate(model_inputs, sampling_params)
responses = [ x.outputs[0].text.replace("<|start_header_id|>assistant<|end_header_id|>", "").strip() for x in result]

Processed prompts: 100%|██████████████████████████| 2/2 [00:02<00:00,  1.42s/it]


In [7]:
responses[0]

"I'm so sorry to hear that you're going through this. Losing a pet is never easy, and it's completely normal to feel overwhelmed with emotions.\n\nFirst of all, please know that you're not alone. Many people have been in your shoes, and it's okay to grieve the loss of your furry friend. Would you like to talk about your dog and the memories you shared with them? Sometimes sharing stories and stories can help process our emotions and feel a bit better.\n\nIt's also important to acknowledge that grief is a journey, and it's okay to take your time. You might need to take things one day at a time, and that's okay. You don't have to rush the process.\n\nAdditionally, have you considered talking to a mental health professional? They can offer you a safe and non-judgmental space to process your emotions and provide you with coping strategies to deal with your grief. Would you like me to help you find some resources or information on how to get started with that?\n\nRemember, you're doing the 

In [7]:
result[0].outputs[0].text

"<|start_header_id|>assistant<|end_header_id|>\n\nI'm so sorry to hear that you're going through this. Losing a pet is never easy, and it's normal to feel sad and emotional.\n\nCan you tell me a little bit more about your dog and what they meant to you? Sometimes sharing your feelings with someone who listens can help you process your emotions.\n\nIt might be helpful to acknowledge that your feelings are valid and that it's okay to grieve the loss of your pet. It's a significant loss, and it's normal to take time to adjust to the new reality.\n\nIf you feel comfortable, what was your dog's personality like? What were some of your favorite memories together?"

In [8]:
responses[0]

"I'm so sorry to hear that you're going through this. Losing a pet is never easy, and it's normal to feel sad and emotional. Can you tell me a little bit more about your dog and what they meant to you? Sometimes sharing your feelings with someone who listens can help you process your emotions. It might be helpful to acknowledge that your feelings are valid and that it's okay to grieve the loss of your pet. It's a significant loss, and it's normal to take time to adjust to the new reality. If you feel comfortable, what was your dog's personality like? What were some of your favorite memories together?"

In [9]:
responses[1]

"Congratulations on your new promotion! That's a tremendous accomplishment! It's understandable that your colleagues might be adjusting to the change, especially if they're used to working under your guidance as an individual contributor. It's usual for people to feel uncertain or even resentful when someone takes a new role, as it can disrupt the workflow and dynamics within the team. As their manager, you have a great opportunity to set a positive tone and build trust with your colleagues. Here are some suggestions to help you navigate this situation: 1. **Communicate openly and transparently**: Hold an open-door meeting or a team-wide discussion to address any concerns or questions your colleagues might have. Be approachable and listen attentively to their thoughts and feelings.\n2. **Emphasize collaboration and teamwork**: Let your team know that your promote doesn't mean you'll be distancing yourself from them. Confirm that you're committed to working closely with them to achieve 

### Install Flash Attention

In [ ]:
# do not have nvcc at beginning, then ask to install nvidia-cuda-toolkit, then the cuda version becomes 11.5 < 11.6
# flash-attention supports 11.6 and above.
# Then update cuda to 11.8 by install nvidia cuda driver, but afterwards the GPU driver returns back to an old version
# Then start to install a new nvidia driver by software & updates app in the ubuntu, like last time, in the updated drivers
# This time we update to 545 though the latest  version should be 550.
# Anyway, let's just use current version to make it run at least. Flash attention works now, good.